# Fine Tuning Transformer for Named Entity Recognition

### Introduction

In this tutorial we will be fine tuning a transformer model for the **Named Entity Recognition** problem. 
This is one of the most common business problems where a given piece of text/sentence/document different entites need to be identified such as: Name, Location, Number, Entity etc.

#### Flow of the notebook

The notebook will be divided into seperate sections to provide a organized walk through for the process used. This process can be modified for individual use cases. The sections are:

1. [Installing packages for preparing the system](#section00)
2. [Importing Python Libraries and preparing the environment](#section01)
3. [Importing and Pre-Processing the domain data](#section02)
4. [Preparing the Dataset and Dataloader](#section03)
5. [Creating the Neural Network for Fine Tuning](#section04)
6. [Fine Tuning the Model](#section05)
7. [Validating the Model Performance](#section06)

#### Technical Details

This script leverages on multiple tools designed by other teams. Details of the tools used below. Please ensure that these elements are present in your setup to successfully implement this script.

 - Data:
	- We are working from a dataset available on [Kaggle](https://www.kaggle.com/)
    - This NER annotated dataset is available at the following [link](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)
    - We will be working with the file `ner.csv` from the dataset. 
    - In the given file we will be looking at the following columns for the purpose of this fine tuning:
        - `sentence_idx` : This is the identifier that the word in the row is part of the same sentence
        - `word` : Word in the sentence
        - `tag` : This is the identifier that is used to identify the entity in the dataset. 
    - The various entites tagged in this dataset are as per below:
        - geo = Geographical Entity
        - org = Organization
        - per = Person
        - gpe = Geopolitical Entity
        - tim = Time indicator
        - art = Artifact
        - eve = Event
        - nat = Natural Phenomenon


 - Language Model Used:
	 - We are using BERT for this project. Hugging face team has created a customized model for token classification, called **BertForTokenClassification**. We will be using it in our custommodel class for training. 
	 - [Blog-Post](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)
     - [Documentation for python](https://huggingface.co/transformers/model_doc/bert.html#bertfortokenclassification)


 - Hardware Requirements:
	 - Python 3.6 and above
	 - Pytorch, Transformers and All the stock Python ML Libraries
	 - TPU enabled setup. This can also be executed over GPU but the code base will need some changes. 


 - Script Objective:
	 - The objective of this script is to fine tune **BertForTokenClassification**` to be able to identify the entites as per the given test dataset. The entities labled in the given dataset are as follows:

In [1]:
import sys

if 'google.colab' in sys.modules:
  from google.colab import drive
  drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/colab_notebooks/kaggle/

/content/drive/MyDrive/colab_notebooks/kaggle


In [3]:
if 'google.colab' in sys.modules:
    !pip install kaggle --upgrade

    import os
    import json
    f = open("kaggle.json", "r")
    json_data = json.load(f)
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]

Requirement already up-to-date: kaggle in /usr/local/lib/python3.7/dist-packages (1.5.12)


In [4]:
if 'google.colab' in sys.modules:
    %cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks
    !pwd

/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks
/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks


<a id='section00'></a>
### Installing packages for preparing the system

We are installing 2 packages for the purposes of TPU execution and f1 metric score calculation respectively
*You can skip this step if you already have these libraries installed in your environment*

In [5]:
!curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!pip -q install seqeval

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  34335      0 --:--:-- --:--:-- --:--:-- 34335
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Uninstalling torch-1.6.0a0+bf2bbd9:
  Successfully uninstalled torch-1.6.0a0+bf2bbd9
Uninstalling torchvision-0.7.0a0+a6073f0:
  Successfully uninstalled torchvision-0.7.0a0+a6073f0
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl...
- [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl...
\
Operation completed over 1 objects/119.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-nightly+20200515-cp37-

<a id='section01'></a>
### Importing Python Libraries and preparing the environment

At this step we will be importing the libraries and modules needed to run our script. Libraries are:
* Pandas
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* BERT Model and Tokenizer

Followed by that we will preapre the device for TPU execeution. This configuration is needed if you want to leverage on onboard TPU. 

In [6]:
# Importing pytorch and the library for TPU execution

import torch
import torch_xla
import torch_xla.core.xla_model as xm

In [7]:
if 'google.colab' in sys.modules:
    !pip install transformers

In [8]:
# Importing stock ml libraries

import numpy as np
import pandas as pd
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel

# Preparing for TPU usage
dev = xm.xla_device()

In [9]:
import gc

<a id='section02'></a>
### Importing and Pre-Processing the domain data

We will be working with the data and preparing for fine tuning purposes. 
*Assuming that the `ner.csv` is already downloaded in your `data` folder*

* Import the file in a dataframe and give it the headers as per the documentation.
* Cleaning the file to remove the unwanted columns.
* We will create a class `SentenceGetter` that will pull the words from the columns and create them into sentences
* Followed by that we will create some additional lists and dict to keep the data that will be used for future processing

In [10]:
cv = 1
dataset_train = pd.read_pickle(f"nb003-annotation-data/nb003_cv{cv}_train.pkl")
dataset_test = pd.read_pickle(f"nb003-annotation-data/nb003_cv{cv}_test.pkl")

In [11]:
dataset_train["isTrain"] = 1
dataset_test["isTrain"] = 0

dataset = pd.concat([dataset_train, dataset_test], axis=0, ignore_index=True)
del dataset_train, dataset_test
gc.collect()

80

In [12]:
dataset["sentence_idx"] = dataset["sentence"] + dataset["sentence#"]

In [13]:
sentence_vals = list(set(dataset["sentence_idx"].values))
sentence2idx = {v: i for i, v in enumerate(sentence_vals)}
del sentence_vals
gc.collect()

88

In [14]:
dataset["sentence_idx"] = dataset["sentence_idx"].apply(lambda x: sentence2idx[x])
del sentence2idx
gc.collect()

0

In [15]:
dataset = dataset[["sentence_idx", "token", "pos", "entity", "isTrain"]].copy()
dataset.rename(columns={"token":"word", "entity":"tag"}, inplace=True)
dataset

,sentence_idx,word,pos,tag,isTrain
0,8671,This,DET,o,1
1,8671,study,NOUN,o,1
2,8671,used,VERB,o,1
3,8671,data,NOUN,o,1
4,8671,from,ADP,o,1
...,...,...,...,...,...
6309716,3219,were,AUX,o,0
6309717,3219,fed,VERB,o,0
6309718,3219,to,ADP,o,0
6309719,3219,the,DET,o,0


In [ ]:
#dataset = pd.read_csv("data/ner_dataset.csv", encoding = "ISO-8859-1", error_bad_lines=False)
#dataset.rename(columns={"Sentence #":"sentence_idx", "Word":"word", "POS":"pos", "Tag":"tag"}, inplace=True)
#dataset["sentence_idx"].fillna(method='ffill', inplace=True)
#dataset["sentence_idx"] = dataset["sentence_idx"].apply(lambda x: float(x[-2:].replace(" ", "")))

In [16]:
num_labels = dataset["tag"].nunique()
print(num_labels)

2


In [ ]:
#df = pd.read_csv("./data/ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)
#dataset=df.drop(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
#       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
#       'next-word', 'prev-iob', 'prev-lemma', 'prev-pos',
#       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
#       'prev-prev-word', 'prev-shape', 'prev-word','shape'],axis=1)

#dataset = pd.read_csv("data/ner_dataset.csv")
#dataset.head()

In [17]:
# Creating a class to pull the words from the columns and create them into sentences

class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w,p,f,t) for w,p,f,t in zip(s["word"].values.tolist(),
                                                       s["pos"].values.tolist(),
                                                       s["isTrain"].values.tolist(),
                                                       s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

getter = SentenceGetter(dataset)

In [18]:
# Creating new lists and dicts that will be used at a later stage for reference and processing

tags_vals = list(set(dataset["tag"].values))
print("tags_vals:", tags_vals)
tag2idx = {t: i for i, t in enumerate(tags_vals)}
sentences = [' '.join([s[0] for s in sent]) for sent in getter.sentences]
poses = [' '.join([s[1] for s in sent]) for sent in getter.sentences]
istrain = [[s[2] for s in sent] for sent in getter.sentences]
labels = [[s[3] for s in sent] for sent in getter.sentences]
labels = [[tag2idx.get(l) for l in lab] for lab in labels]

tags_vals: ['o', 'o-dataset']


<a id='section03'></a>
### Preparing the Dataset and Dataloader

We will start with defining few key variables that will be used later during the training/fine tuning stage.
Followed by creation of Dataset class - This defines how the text is pre-processed before sending it to the neural network. We will also define the Dataloader that will feed  the data in batches to the neural network for suitable training and processing. 
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *CustomDataset* Dataset Class
- This class is defined to accept the `tokenizer`, `sentences` and `labels` as input and generate tokenized output and tags that is used by the BERT model for training. 
- We are using the BERT tokenizer to tokenize the data in the `sentences` list for encoding. 
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/bert.html#berttokenizer)
- `tags` is the encoded entity from the annonated dataset. 
- The *CustomDataset* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training. 

#### Dataloader
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [19]:
# Defining some key variables that will be used later on in the training

MAX_LEN = 290 #200
TRAIN_BATCH_SIZE = 32*1
VALID_BATCH_SIZE = 16*1
EPOCHS = 5
LEARNING_RATE = 2e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [20]:
class CustomDataset(Dataset):
    def __init__(self, tokenizer, sentences, poses, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.poses = poses
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        sentence = str(self.sentences[index])
        pos = str(self.poses[index])
        inputs = self.tokenizer.encode_plus(
            sentence,
            #### RIOW
            #None,
            pos,
            #### RIOWRIOW
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        label = self.labels[index]
        label.extend([4]*self.max_len)
        label=label[:self.max_len]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'tags': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [ ]:
#[sentences[i] for i in np.where(istrain==1)[0]]
#[labels[i] for i in np.where(istrain==1)[0]]

In [21]:
# Creating the dataset and dataloader for the neural network

#### RIOW
#train_percent = 0.8
#train_size = int(train_percent*len(sentences))
#### RIOWRIOW

# train_dataset=df.sample(frac=train_size,random_state=200).reset_index(drop=True)
# test_dataset=df.drop(train_dataset.index).reset_index(drop=True)


#### RIOW
istrain = [ist[0] for ist in istrain]
istrain = np.array(istrain)

#train_sentences = sentences[0:train_size]
#train_labels = labels[0:train_size]
train_sentences = [sentences[i] for i in np.where(istrain==1)[0]]
train_poses = [poses[i] for i in np.where(istrain==1)[0]]
train_labels = [labels[i] for i in np.where(istrain==1)[0]]

#test_sentences = sentences[train_size:]
#test_labels = labels[train_size:]
test_sentences = [sentences[i] for i in np.where(istrain==0)[0]]
test_poses = [poses[i] for i in np.where(istrain==0)[0]]
test_labels = [labels[i] for i in np.where(istrain==0)[0]]
#### RIOWRIOW

print("FULL Dataset: {}".format(len(sentences)))
print("TRAIN Dataset: {}".format(len(train_sentences)))
print("TEST Dataset: {}".format(len(test_sentences)))

training_set = CustomDataset(tokenizer, train_sentences, train_poses, train_labels, MAX_LEN)
testing_set = CustomDataset(tokenizer, test_sentences, train_poses, test_labels, MAX_LEN)

FULL Dataset: 23337
TRAIN Dataset: 18670
TEST Dataset: 4667


In [22]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `BERTClass`. 
 - This network will have the `BertForTokenClassification` model. 
 - The data will be fed to the `BertForTokenClassification` as defined in the dataset. 
 - Final layer outputs is what will be used to calcuate the loss and to determine the accuracy of models prediction. 
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference. 
 
#### Loss Function and Optimizer
 - `Optimizer` is defined in the next cell.
 - We do not define any `Loss function` since the specified model already outputs `Loss` for a given input. 
 - `Optimizer` is used to update the weights of the neural network to improve its performance.
 
#### Further Reading
- You can refer to my [Pytorch Tutorials](https://github.com/abhimishra91/pytorch-tutorials) to get an intuition of Loss Function and Optimizer.
- [Pytorch Documentation for Loss Function](https://pytorch.org/docs/stable/nn.html#loss-functions)
- [Pytorch Documentation for Optimizer](https://pytorch.org/docs/stable/optim.html)
- Refer to the links provided on the top of the notebook to read more about `BertForTokenClassification`. 

In [23]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        #self.l1 = transformers.BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=num_labels)
        #self.l1 = transformers.BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=18)
        self.l1 = transformers.BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=num_labels)
        # self.l2 = torch.nn.Dropout(0.3)
        # self.l3 = torch.nn.Linear(768, 200)
    
    def forward(self, ids, mask, labels):
        output_1= self.l1(ids, mask, labels = labels)
        # output_2 = self.l2(output_1[0])
        # output = self.l3(output_2)
        return output_1

    #def save_pretrained(self, path):
    #    self.l1.save_pretrained(path)

In [24]:
model = BERTClass()
model.to(dev)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

BERTClass(
  (l1): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
       

In [25]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

<a id='section05'></a>
### Fine Tuning the Model

After all the effort of loading and preparing the data and datasets, creating the model and defining its loss and optimizer. This is probably the easier steps in the process. 

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network. 

Following events happen in this function to fine tune the neural network:
- The dataloader passes data to the model based on the batch size. 
- Subsequent output from the model and the actual category are compared to calculate the loss. 
- Loss value is used to optimize the weights of the neurons in the network.
- After every 500 steps the loss value is printed in the console.

As you can see just in 1 epoch by the final step the model was working with a miniscule loss of 0.08503091335296631 i.e. the output is extremely close to the actual output.

In [26]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(dev, dtype = torch.long)
        mask = data['mask'].to(dev, dtype = torch.long)
        targets = data['tags'].to(dev, dtype = torch.long)

        loss = model(ids, mask, labels = targets)[0]

        # optimizer.zero_grad()
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        xm.optimizer_step(optimizer)
        xm.mark_step() 

In [27]:
for epoch in range(EPOCHS):
#for epoch in range(1):
    train(epoch)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  0.6684166789054871
Epoch: 0, Loss:  0.004678213037550449
Epoch: 1, Loss:  0.009800302796065807
Epoch: 1, Loss:  0.0074195656925439835
Epoch: 2, Loss:  0.0040416172705590725
Epoch: 2, Loss:  0.01089267898350954
Epoch: 3, Loss:  0.01759752817451954
Epoch: 3, Loss:  0.004667059984058142
Epoch: 4, Loss:  0.00938216969370842
Epoch: 4, Loss:  0.005451881792396307


<a id='section06'></a>
### Validating the Model

During the validation stage we pass the unseen data(Testing Dataset) to the model. This step determines how good the model performs on the unseen data. 

This unseen data is the 30% of `ner.csv` which was seperated during the Dataset creation stage. 
During the validation stage the weights of the model are not updated. Only the final output is compared to the actual value. This comparison is then used to calcuate the accuracy of the model. 

The metric used for measuring the performance of model for these problem statements is called F1 score. We will create a helper function for helping us with f1 score calcuation and also import a library for the same. `seqeval`

In [28]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    flat_preds = np.argmax(preds, axis=2).flatten()
    flat_labels = labels.flatten()
    return np.sum(flat_preds == flat_labels)/len(flat_labels)

In [29]:
def valid(model, testing_loader):
    model.eval()
    eval_loss = 0; eval_accuracy = 0
    n_correct = 0; n_wrong = 0; total = 0
    predictions , true_labels = [], []
    nb_eval_steps, nb_eval_examples = 0, 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(dev, dtype = torch.long)
            mask = data['mask'].to(dev, dtype = torch.long)
            targets = data['tags'].to(dev, dtype = torch.long)

            output = model(ids, mask, labels=targets)
            loss, logits = output[:2]
            logits = logits.detach().cpu().numpy()
            
            label_ids = targets.to('cpu').numpy()
            predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
            true_labels.append(label_ids)
            
            accuracy = flat_accuracy(logits, label_ids)
            eval_loss += loss.mean().item()
            eval_accuracy += accuracy
            nb_eval_examples += ids.size(0)
            nb_eval_steps += 1
        eval_loss = eval_loss/nb_eval_steps
        print("Validation loss: {}".format(eval_loss))
        print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
        
        #pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
        #valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
        #print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

    return predictions, true_labels

In [30]:
# To get the results on the validation set. This data is not seen by the model

#valid(model, testing_loader)
predictions, true_labels = valid(model, testing_loader)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Validation loss: 0.010680772218221722
Validation Accuracy: 0.9305077001116506


In [ ]:
37638 / 16

2352.375

In [31]:
# TEST Dataset: 37638 == num_obs
# predictions: (num_obs, max_len)
len(predictions)

4667

In [32]:
len(predictions[0])

290

In [33]:
# pred_tags: (num_obs * max_len == 7527600, )
pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]

In [34]:
set(pred_tags)
# 一応2ラベルになっている

{'o', 'o-dataset'}

In [35]:
len([p for p in pred_tags if p=="o"]) / len(pred_tags)

0.9997251427853675

In [36]:
len([p for p in pred_tags if p=="o-dataset"])

372

In [37]:
len(predictions) * len(predictions[0])

1353430

In [38]:
len(pred_tags)

1353430

In [39]:
# Validation dataset
tmp = dataset[dataset["isTrain"]==0]

In [40]:
res = dict()
for i, idx in enumerate(tmp["sentence_idx"].unique()):
    seq = tmp[tmp["sentence_idx"]==idx]["word"].values
    seq = np.pad(seq, (0, MAX_LEN-len(seq)), 'constant', constant_values="[PAD]")
    res[idx] = (seq, pred_tags[i:i+MAX_LEN])

In [41]:
tmp.drop_duplicates(subset="sentence_idx", keep='first', inplace=True)
tmp.drop(["tag", "word"], axis=1, inplace=True)
tmp["pred"]=""
tmp.reset_index(drop=True, inplace=True)

for i, row in tmp.iterrows():
    #print(row[0])
    idx = int(row[0])
    cond = np.where(np.array(res[idx][1])=='o-dataset')[0]
    pred = " ".join(list(np.array(res[idx][0])[cond]))
    #row["pred"] = pred
    tmp.at[i, 'pred'] = pred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [42]:
tmp

,sentence_idx,pos,isTrain,pred
0,10000,NOUN,0,
1,16074,PRON,0,
2,5735,NOUN,0,
3,15634,PROPN,0,
4,15724,VERB,0,
...,...,...,...,...
4662,2491,NOUN,0,
4663,3525,CCONJ,0,
4664,2606,NOUN,0,
4665,16261,DET,0,


In [43]:
tmp = tmp[tmp["pred"]!=""]
tmp.reset_index(drop=True, inplace=True)

In [44]:
tmp

,sentence_idx,pos,isTrain,pred
0,5242,NOUN,0,5
1,14440,NOUN,0,1
2,7216,ADJ,0,to
3,1842,NOUN,0,rank
4,5556,DET,0,the
...,...,...,...,...
285,17742,DET,0,Neuroinformatics
286,5122,NOUN,0,modeling
287,7779,NOUN,0,protocol
288,6464,NOUN,0,from


In [45]:
tmp["pred"][0]

'5'

In [46]:
tmp["pred"][1]

'1'

In [ ]:
tmp["pred"][2]

'site throughout the'

In [ ]:
tmp["pred"][3]

'h, determined by a'

In [ ]:
tmp["pred"][4]

'additive predictors that provided the'

In [ ]:
tmp["pred"][5]

'In a second approach, after examining'

In [ ]:
tmp["pred"][6]

'of information (for instance, radio-telemetry based mark-resight'

In [ ]:
tmp["pred"][7]

'of seals counted within each survey zone are'

In [ ]:
tmp["pred"][589]

'20,'

In [47]:
tmp["pred"].sample(10)

211            the
144         method
63             the
177    convolution
265              a
249        imaging
71             edu
47         finding
203        healthy
164          could
Name: pred, dtype: object

In [ ]:
# true_labels: (num_obs/batch_size, batch_size, max_len)
len(true_labels)

2353

In [ ]:
true_labels[1].shape

(16, 290)

In [48]:
tags_vals[1]

'o-dataset'

In [49]:
tmp = np.array([l_ii for l in true_labels for l_i in l for l_ii in l_i])

In [50]:
np.unique(tmp)

array([0, 1, 4])

In [51]:
np.where(tmp==0)[0].shape

(1259447,)

In [52]:
np.where(tmp==1)[0].shape

(2473,)

In [53]:
np.where(tmp==4)[0].shape

(91510,)

In [54]:
valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]

IndexError: ignored

In [55]:
print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

NameError: ignored

# Save the model

In [56]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
#print("Optimizer's state_dict:")
#for var_name in optimizer.state_dict():
#    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
l1.bert.embeddings.position_ids 	 torch.Size([1, 512])
l1.bert.embeddings.word_embeddings.weight 	 torch.Size([28996, 768])
l1.bert.embeddings.position_embeddings.weight 	 torch.Size([512, 768])
l1.bert.embeddings.token_type_embeddings.weight 	 torch.Size([2, 768])
l1.bert.embeddings.LayerNorm.weight 	 torch.Size([768])
l1.bert.embeddings.LayerNorm.bias 	 torch.Size([768])
l1.bert.encoder.layer.0.attention.self.query.weight 	 torch.Size([768, 768])
l1.bert.encoder.layer.0.attention.self.query.bias 	 torch.Size([768])
l1.bert.encoder.layer.0.attention.self.key.weight 	 torch.Size([768, 768])
l1.bert.encoder.layer.0.attention.self.key.bias 	 torch.Size([768])
l1.bert.encoder.layer.0.attention.self.value.weight 	 torch.Size([768, 768])
l1.bert.encoder.layer.0.attention.self.value.bias 	 torch.Size([768])
l1.bert.encoder.layer.0.attention.output.dense.weight 	 torch.Size([768, 768])
l1.bert.encoder.layer.0.attention.output.dense.bias 	 torch.Size([768])
l1.bert.encoder.

In [ ]:
"""
output_model = './models/model_xlnet_mid.pth'

# save
def save(model, optimizer):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)

save(model, optimizer)
"""

In [57]:
folder = "localnb001-transformers-ner"
!mkdir {folder}
PATH = f"bert-base-cased-ner-cv{cv}.pth"

def save(model, optimizer, folder, path):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, "./"+folder+"/"+path)

save(model, optimizer, folder, PATH)

mkdir: cannot create directory ‘localnb001-transformers-ner’: File exists


In [ ]:
!ls -l {folder}/

total 4335433
-rw------- 1 root root  647113465 Apr 24 02:26 bert-base-cased-ner-cv1.bin
-rw------- 1 root root  647113465 Apr 24 06:33 bert-base-cased-ner-cv1.pt
-rw------- 1 root root 1870415517 Apr 27 04:13 bert-base-cased-ner-cv1.pth
-rw------- 1 root root  404400730 Apr 24 07:26 bert-base-cased.tar.gz
-rw------- 1 root root     213450 Nov 30  2018 bert-base-cased-vocab.txt
-rw------- 1 root root        614 Apr 25 02:11 config.json
-rw------- 1 root root        121 Apr 23 06:52 dataset-metadata.json
-rw------- 1 root root  647118093 Apr 25 02:11 pytorch_model.bin
-rw------- 1 root root       5116 Apr 22 23:29 pytorch-xla-env-setup.py
-rw------- 1 root root   95368646 Apr 22 23:29 torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl
-rw------- 1 root root    2443535 Apr 22 23:29 torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl
-rw------- 1 root root  125287043 Apr 22 23:29 torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl


In [ ]:
#folder = "localnb001-transformers-ner"
#!mkdir {folder}
#PATH = f"bert-base-cased-ner-cv{cv}.pth"

#torch.save(model.state_dict(), PATH)
#torch.save(model.state_dict(), PATH)
#!mv {PATH} {folder}/{PATH}

In [ ]:
# folder = "localnb001-transformers-ner"
# !mkdir {folder}

# # huggingface fine-tuned model (pre-trained) save method is different from standard PyTroch save method
# model.save_pretrained(folder)
# # reference: https://huggingface.co/transformers/model_sharing.html

mkdir: cannot create directory ‘localnb001-transformers-ner’: File exists


# Upload to Kaggle

In [ ]:
#folder = "localnb001-transformers-ner"

In [58]:
#!kaggle datasets version -p {folder} -m "renamed fine-tuned model (.pt) added"
#!kaggle datasets version -p {folder} -m "huggingface's fine-tuned model added"
!kaggle datasets version -p {folder} -m "[Update] POS tag added as iput"

Starting upload for file bert-base-cased-vocab.txt
100% 208k/208k [00:02<00:00, 90.0kB/s]
Upload successful: bert-base-cased-vocab.txt (208KB)
Starting upload for file pytorch-xla-env-setup.py
100% 5.00k/5.00k [00:00<00:00, 27.0kB/s]
Upload successful: pytorch-xla-env-setup.py (5KB)
Starting upload for file torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl
100% 91.0M/91.0M [00:04<00:00, 21.1MB/s]
Upload successful: torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl (91MB)
Starting upload for file torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl
100% 119M/119M [00:04<00:00, 26.8MB/s]
Upload successful: torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl (119MB)
Starting upload for file torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl
100% 2.33M/2.33M [00:02<00:00, 1.21MB/s]
Upload successful: torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl (2MB)
Starting upload for file bert-base-cased-ner-cv1.bin
100% 617M/617M [00:16<00:00, 40.2MB/s]
Upload successful: bert-

In [ ]:
#!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt
#!mv bert-base-cased-vocab.txt {folder}/bert-base-cased-vocab.txt
#!kaggle datasets version -p {folder} -m "pre-trained BertTokenizer added"

--2021-04-23 14:36:18--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.153.14
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.153.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 213450 (208K) [text/plain]
Saving to: ‘bert-base-cased-vocab.txt’

bert-base-cased-voc 100%[===================>] 208.45K  --.-KB/s    in 0.1s    

2021-04-23 14:36:18 (1.67 MB/s) - ‘bert-base-cased-vocab.txt’ saved [213450/213450]



In [ ]:
#!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz
#!mv bert-base-cased.tar.gz {folder}/bert-base-cased.tar.gz
#!kaggle datasets version -p {folder} -m "pre-trained BertForTokenClassification added"

--2021-04-24 07:26:54--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.144.125
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.144.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 404400730 (386M) [application/x-tar]
Saving to: ‘bert-base-cased.tar.gz’

bert-base-cased.tar 100%[===================>] 385.67M  55.9MB/s    in 6.7s    

2021-04-24 07:27:01 (57.4 MB/s) - ‘bert-base-cased.tar.gz’ saved [404400730/404400730]

Starting upload for file bert-base-cased-vocab.txt
100% 208k/208k [00:01<00:00, 156kB/s]
Upload successful: bert-base-cased-vocab.txt (208KB)
Starting upload for file pytorch-xla-env-setup.py
100% 5.00k/5.00k [00:01<00:00, 4.79kB/s]
Upload successful: pytorch-xla-env-setup.py (5KB)
Starting upload for file torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl
100% 91.0M/91.0M [00:02<00:00, 42.0MB/s]
Upload successful: torch-nightly+20200515-cp37-cp37m-l

In [ ]:
!kaggle datasets init -p {folder}
# referene: https://kaeru-nantoka.hatenablog.com/entry/2020/01/17/015551

with open(f"{folder}/dataset-metadata.json", "r") as jsonFile:
    data = json.load(jsonFile)

data["id"] = f"riow1983/{folder}"
data["title"] = folder

with open(f"{folder}/dataset-metadata.json", "w") as jsonFile:
    json.dump(data, jsonFile)

!kaggle datasets create -p {folder}

Data package template written to: localnb001-transformers-ner/dataset-metadata.json
Starting upload for file model_initial.pth
100% 617M/617M [00:08<00:00, 80.0MB/s]
Upload successful: model_initial.pth (617MB)
Starting upload for file bert-base-cased-ner-cv1.pth
100% 617M/617M [00:10<00:00, 62.4MB/s]
Upload successful: bert-base-cased-ner-cv1.pth (617MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/riow1983/localnb001-transformers-ner
